<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

Using TensorFlow backend.


In [0]:
train_data_dir = '/content/drive/My Drive/dataset'

In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.3)
batch_size = 32

# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(638, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 3297 images belonging to 2 classes.
Found 1412 images belonging to 2 classes.


In [0]:
epochs = 20
train_samples = 3297
validation_samples = 1412

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x=Dense(1024,activation='relu')(x) #dense layer 2
#x=Dense(512,activation='relu')(x) #dense layer 3
#x = Dropout(0.5)(x)
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
model=Model(inputs=base_model.input,outputs=preds)
for i,layer in enumerate(model.layers):
  print(i,layer.name)

for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

0 input_8
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/20
32/32 [==============================] - 200s 6s/step - loss: 0.5809 - acc: 0.7607 - val_loss: 0.7498 - val_acc: 0.4639

Epoch 00001: val_acc improved from -inf to 0.46387, saving model to Breast.h5
Epoch 2/20
32/32 [==============================] - 119s 4s/step - loss: 0.4175 - acc: 0.7996 - val_loss: 1.0640 - val_acc: 0.4880

Epoch 00002: val_acc improved from 0.46387 to 0.48795, saving model to Breast.h5
Epoch 3/20
32/32 [==============================] - 130s 4s/step - loss: 0.3461 - acc: 0.8516 - val_loss: 2.0083 - val_acc: 0.5000

Epoch 00003: val_acc improved from 0.48795 to 0.50000, saving model to Breast.h5
Epoch 4/20
32/32 [==============================] - 145s 5s/step - loss: 0.3001 - acc: 0.8730 - val_loss: 1.1889 - val_acc: 0.5039

Epoch 00004: val_acc improved from 0.50000 to 0.50391, saving model to Breast.h5
Epoch 5/20
32/32 [==============================] - 140s 4s/step - loss: 0.3096 - acc: 0.8633 - val_loss: 2.9763 - val_acc: 0.4910

Epoch 00005: val_ac

In [0]:
  model.save("Breast.model")

In [0]:
import cv2
import tensorflow as tf
import math
#from google.colab.patches import cv2_imshow

CATEGORIES = ["normal", "cancer"]


def prepare(filepath):
    #IMG_SIZE = 220 # 50 in txt-based
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (224,224))

    return new_array.reshape(-1, 224,224, 3)


model = tf.keras.models.load_model("Breast.model")

prediction = model.predict([prepare('/content/drive/My Drive/dataset/NORMAL/normal_04-A_0475_1.LEFT_MLO.LJPEG.1.jpg')])
#pred_percent = math.floor(prediction[0][0]*100)
#print(pred_percent,"%")
#print(prediction)  # will be a list in a list.
#print(CATEGORIES[int(prediction[0][0])])

if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(prediction)
print(result*100)
print(CATEGORIES[int(result)])

[[0.94623065 0.05376933]]
94.62306499481201
normal


In [0]:
normal = 2342
cancer = 2367

if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(result)

0.50929433


In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen1 = ImageDataGenerator(rescale=1. / 255)
batch_size = 32
train_data_dir1 = '/content/drive/My Drive/dataset1/' 
# Retrieve images and their classes for train and validation sets
train_generator1 = datagen1.flow_from_directory(
        train_data_dir1,
        target_size=(224, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

Found 2367 images belonging to 1 classes.
